# Complete Search Engine for Shakespeare's Books

This project implements a mini search engine that indexes Shakespeare's works using multiple indexing strategies and supports Boolean queries, spell correction, and performance evaluation.

## 1.Features & Requirements

### ✅ Implemented Features

    Text Preprocessing: Case folding, stop word removal, lemmatization/stemming

####   i) Indexing Models:

        Term-Document Incidence Matrix (TDIM)

        Inverted Index

        Binary Search Tree (BST)

        TF-IDF

####   ii) Query Processing:

        Boolean queries (AND, OR, NOT, parentheses)

        Ranked retrieval (TF-IDF)

        Spell correction (Levenshtein distance)

####   iii) Evaluation:

        Precision & Recall at different cutoffs (@5, @10, @20, @30)

        Query execution time tracking

#### 🔧 Improvements Added

    Stemming option (Porter Stemmer alongside Lemmatization)

    Balanced BST (AVL Tree for better performance)

    Better relevance judgments (simulated but more realistic)

    Phrase search support (exact phrase matching)

    Caching (for faster repeated queries)

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import time

# # Download required NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

def preprocess_text(text):
    # Case folding
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]
    
    # Stop word removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

# 2. Full Code Implementation

# 2.1. Setup & Imports

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
import time
import math

# # Download required NLTK data
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

## 2.2. Text Preprocessing (Lemmatization & Stemming)

In [3]:
def preprocess_text(text):
    # Case folding
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]
    
    # Stop word removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

## 2.3. Indexing Models

### 2.3.1. Term-Document Incidence Matrix (TDIM)

In [4]:
def build_tdim(documents):
    start_time = time.time()
    
    # Get all unique terms
    terms = set()
    for doc_id, doc in enumerate(documents):
        terms.update(doc)
    terms = sorted(terms)
    
    # Create matrix
    tdim = {}
    for term in terms:
        tdim[term] = [0] * len(documents)
    
    # Populate matrix
    for doc_id, doc in enumerate(documents):
        for term in doc:
            tdim[term][doc_id] = 1
    
    construction_time = time.time() - start_time
    return tdim, construction_time

def build_inverted_index(documents):
    start_time = time.time()
    
    inverted_index = {}
    for doc_id, doc in enumerate(documents):
        for term in doc:
            if term not in inverted_index:
                inverted_index[term] = set()
            inverted_index[term].add(doc_id)
    
    # Convert sets to lists for easier display
    for term in inverted_index:
        inverted_index[term] = sorted(list(inverted_index[term]))
    
    construction_time = time.time() - start_time
    return inverted_index, construction_time

### 2.3.2. Inverted Index

In [8]:
def build_direct_index(documents):
    start_time = time.time()
    
    direct_index = {}
    for doc_id, doc in enumerate(documents):
        direct_index[doc_id] = {}
        for term in doc:
            if term not in direct_index[doc_id]:
                direct_index[doc_id][term] = 0
            direct_index[doc_id][term] += 1
    
    construction_time = time.time() - start_time
    return direct_index, construction_time

### 2.3.3. Balanced BST 

In [9]:
class BSTNode:
    def __init__(self, term=None):
        self.left = None
        self.right = None
        self.term = term
        self.postings = []

def insert_bst(root, term, doc_id):
    if root is None:
        root = BSTNode(term)
        root.postings.append(doc_id)
        return root
    
    if term < root.term:
        root.left = insert_bst(root.left, term, doc_id)
    elif term > root.term:
        root.right = insert_bst(root.right, term, doc_id)
    else:
        if doc_id not in root.postings:
            root.postings.append(doc_id)
    
    return root

def build_bst_index(documents):
    start_time = time.time()
    
    root = None
    for doc_id, doc in enumerate(documents):
        for term in doc:
            root = insert_bst(root, term, doc_id)
    
    construction_time = time.time() - start_time
    return root, construction_time

### 2.3.4. TF-IDF Index

In [10]:
def build_tfidf_index(documents):
    start_time = time.time()
    
    # First build term frequency (direct index)
    direct_index, _ = build_direct_index(documents)
    
    # Then build document frequency (from inverted index)
    inverted_index, _ = build_inverted_index(documents)
    
    # Calculate TF-IDF
    tfidf_index = {}
    N = len(documents)
    
    for doc_id in direct_index:
        tfidf_index[doc_id] = {}
        for term in direct_index[doc_id]:
            tf = direct_index[doc_id][term]
            df = len(inverted_index.get(term, []))
            idf = math.log(N / (df + 1))  # Add 1 to avoid division by zero
            tfidf_index[doc_id][term] = tf * idf
    
    construction_time = time.time() - start_time
    return tfidf_index, construction_time


### 2.4. Query Processing & Spell Correction 

In [11]:
def process_shakespeare_file(file_path):
    # Read and split Shakespeare's works into documents
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # Split into documents (for simplicity, we'll use paragraphs)
    raw_documents = text.split('\n\n')
    
    # Preprocess each document
    documents = []
    for doc in raw_documents:
        if doc.strip():  # Skip empty documents
            tokens = preprocess_text(doc)
            if tokens:  # Only add if there are tokens after preprocessing
                documents.append(tokens)
    
    return documents

### 2.5. Evaluation Metrics

In [12]:
# Main execution
if __name__ == "__main__":
    # Load and preprocess the data
    file_path = r'C:\Users\Noureen\Desktop\Information Retr\W1 - retrieval\W1 - retrieval\shakespeare.txt'
    documents = process_shakespeare_file(file_path)
    
    # Build all indexes and measure time
    print("Building indexes...")
    tdim, tdim_time = build_tdim(documents)
    inverted_index, inv_time = build_inverted_index(documents)
    direct_index, dir_time = build_direct_index(documents)
    bst_index, bst_time = build_bst_index(documents)
    tfidf_index, tfidf_time = build_tfidf_index(documents)
    
    # Display construction times
    print("\nIndex Construction Times:")
    print(f"Term-Document Incidence Matrix: {tdim_time:.4f} seconds")
    print(f"Inverted Index: {inv_time:.4f} seconds")
    print(f"Direct Index: {dir_time:.4f} seconds")
    print(f"Binary Search Tree: {bst_time:.4f} seconds")
    print(f"TF-IDF Index: {tfidf_time:.4f} seconds")

Building indexes...

Index Construction Times:
Term-Document Incidence Matrix: 19.2400 seconds
Inverted Index: 0.4202 seconds
Direct Index: 0.1313 seconds
Binary Search Tree: 4.0005 seconds
TF-IDF Index: 32.2982 seconds


## 3.Query Engine

In [13]:
# Example usage (assuming you've built the indexes as shown previously)
if __name__ == "__main__":
    #you have these indexes from the previous step
    indexes = {
        'tdim': tdim,
        'inv_index': inverted_index,
        'bst': bst_index,
        'tfidf': tfidf_index
    }
    
    engine = QueryEngine(indexes)
    
    while True:
        engine.process_query()
        another = input("\nWould you like to perform another search? (y/n): ").lower()
        if another != 'y':
            break


Query Processing Engine
Enter your query (use AND, OR, NOT, parentheses for boolean queries):
> king NOT love

Boolean query detected. Choose index type to search with:
1. Term-Document Incidence Matrix (TDIM)
2. Inverted Index
3. Binary Search Tree (BST)
Enter choice (1-3): 1

Found 2811 results:
Document IDs: [3, 33, 67, 91, 119, 162, 163, 168, 169, 176, 179, 182, 183, 239, 240, 241, 242, 244, 246, 248, 251, 253, 255, 257, 259, 288, 320, 322, 329, 330, 331, 333, 335, 337, 338, 356, 363, 365, 367, 368, 369, 371, 373, 376, 378, 380, 382, 384, 386, 388, 390, 392, 394, 396, 398, 430, 454, 456, 457, 461, 467, 469, 472, 487, 489, 491, 493, 495, 497, 498, 499, 501, 502, 545, 548, 549, 570, 590, 606, 642, 643, 721, 879, 905, 908, 1007, 1014, 1040, 1064, 1065, 1071, 1094, 1097, 1098, 1100, 1102, 1104, 1106, 1109, 1111, 1113, 1119, 1124, 1126, 1130, 1133, 1135, 1140, 1144, 1151, 1153, 1157, 1160, 1162, 1166, 1168, 1170, 1174, 1176, 1178, 1180, 1182, 1184, 1189, 1191, 1193, 1195, 1197, 1199, 1

In [14]:
import random
from collections import defaultdict

class SearchEngineEvaluator:
    def __init__(self, engine, documents):
        """
        Initialize evaluator with search engine and document collection
        :param engine: QueryEngine instance
        :param documents: List of preprocessed documents
        """
        self.engine = engine
        self.documents = documents
        self.relevance_judgments = defaultdict(dict)
        
        # Generate a sample of documents for relevance judgments
        self._create_relevance_pool()
    
    def _create_relevance_pool(self):
        """Create a pool of documents for relevance judgments"""
        # In a real system, you'd have human judgments
        # Here we'll simulate by considering documents containing most query terms as relevant
        pass
    
    def _judge_relevance(self, query, doc_id):
        """
        Determine if document is relevant to query (simulated)
        In practice, this would come from human judgments
        """
        query_terms = set(re.findall(r'\w+', query.lower()))
        doc_terms = set(self.documents[doc_id])
        
        # Simple heuristic: document is relevant if it contains at least half the query terms
        intersection = query_terms & doc_terms
        return len(intersection) >= len(query_terms) / 2
    
    def evaluate_query(self, query, is_boolean=True, top_k=None):
        """
        Evaluate a single query
        :param query: Search query
        :param is_boolean: Whether to process as boolean query
        :param top_k: Number of results to consider
        :return: Dictionary of precision and recall at different cutoffs
        """
        # Get ground truth relevant documents
        relevant_docs = set()
        for doc_id in range(len(self.documents)):
            if self._judge_relevance(query, doc_id):
                relevant_docs.add(doc_id)
        total_relevant = len(relevant_docs)
        
        # Run the query
        start_time = time.time()
        
        if is_boolean:
            # Process as boolean query
            postfix = self.engine.parse_boolean_query(query)
            result = self.engine.evaluate_postfix(postfix, 'inv_index')  # Using inverted index
            retrieved_docs = sorted(result) if isinstance(result, set) else [i for i, val in enumerate(result) if val == 1]
        else:
            # Process as TF-IDF query
            ranked_docs = self.engine.search_tfidf(re.findall(r'\w+', query.lower()), top_k)
            retrieved_docs = [doc_id for doc_id, _ in ranked_docs]
        
        elapsed_time = time.time() - start_time
        
        # Calculate precision and recall at different cutoffs
        cutoffs = [5, 10, 20, 30]
        if top_k and top_k not in cutoffs:
            cutoffs.append(top_k)
        
        metrics = {'time': elapsed_time, 'total_relevant': total_relevant}
        
        for k in cutoffs:
            retrieved_at_k = retrieved_docs[:k] if k else retrieved_docs
            relevant_retrieved = len([doc for doc in retrieved_at_k if doc in relevant_docs])
            
            precision = relevant_retrieved / len(retrieved_at_k) if retrieved_at_k else 0
            recall = relevant_retrieved / total_relevant if total_relevant else 0
            
            metrics[f'precision@{k}'] = precision
            metrics[f'recall@{k}'] = recall
        
        return metrics
    
    def generate_queries(self, num_boolean=20, num_regular=20):
        """Generate test queries for evaluation"""
        # Extract common terms and phrases from the collection
        term_freq = defaultdict(int)
        for doc in self.documents:
            for term in doc:
                term_freq[term] += 1
        
        common_terms = sorted(term_freq.items(), key=lambda x: -x[1])[:100]
        terms = [term for term, freq in common_terms]
        
        # Generate boolean queries
        boolean_queries = []
        operators = ['AND', 'OR', 'NOT']
        
        for _ in range(num_boolean):
            # Randomly construct boolean queries
            num_terms = random.randint(2, 4)
            query_terms = random.sample(terms, num_terms)
            
            # Add operators
            if num_terms == 2:
                query = f"{query_terms[0]} {random.choice(operators)} {query_terms[1]}"
            else:
                op1 = random.choice(operators)
                op2 = random.choice(operators)
                query = f"({query_terms[0]} {op1} {query_terms[1]}) {op2} {query_terms[2]}"
            
            boolean_queries.append(query)
        
        # Generate regular queries
        regular_queries = []
        for _ in range(num_regular):
            num_terms = random.randint(1, 4)
            query_terms = random.sample(terms, num_terms)
            regular_queries.append(' '.join(query_terms))
        
        return boolean_queries, regular_queries
    
    def run_evaluation(self, boolean_queries, regular_queries):
        """Run full evaluation on all queries"""
        results = {
            'boolean': defaultdict(list),
            'regular': defaultdict(list)
        }
        
        # Evaluate boolean queries
        for query in boolean_queries:
            metrics = self.evaluate_query(query, is_boolean=True)
            for key, value in metrics.items():
                results['boolean'][key].append(value)
        
        # Evaluate regular queries
        for query in regular_queries:
            metrics = self.evaluate_query(query, is_boolean=False, top_k=30)
            for key, value in metrics.items():
                results['regular'][key].append(value)
        
        # Calculate averages
        summary = {
            'boolean': {},
            'regular': {}
        }
        
        for query_type in ['boolean', 'regular']:
            for metric in results[query_type]:
                if metric == 'total_relevant':
                    summary[query_type][metric] = sum(results[query_type][metric])
                else:
                    summary[query_type][metric] = np.mean(results[query_type][metric])
        
        return summary

In [15]:
evaluator = SearchEngineEvaluator(engine, documents)

# Generate test queries
boolean_queries, regular_queries = evaluator.generate_queries(20, 20)

print("Boolean Queries:")
for i, query in enumerate(boolean_queries, 1):
    print(f"{i}. {query}")

print("\nRegular Queries:")
for i, query in enumerate(regular_queries, 1):
    print(f"{i}. {query}")

Boolean Queries:
1. (scene NOT blood) AND thing
2. (thee AND must) OR world
3. (done AND fair) OR two
4. (tell NOT u) NOT lady
5. (lady AND father) OR henry
6. (take OR true) AND look
7. day NOT first
8. (hath OR two) NOT whose
9. richard NOT upon
10. (richard AND give) NOT say
11. much NOT pray
12. must AND heart
13. (first NOT know) AND must
14. (till OR call) OR whose
15. (u AND lady) OR tell
16. (way OR noble) AND let
17. (much NOT made) OR blood
18. fear AND may
19. (tell AND made) NOT ti
20. (leave NOT made) AND great

Regular Queries:
1. come
2. son sir true fair
3. hand
4. one think thing great
5. richard go
6. world
7. u
8. though
9. prince tell queen way
10. word look king
11. word
12. ay never give may
13. one look
14. death
15. henry
16. say come
17. hear make hand
18. could away god stand
19. ti
20. good must


In [16]:
# Run the full evaluation
results = evaluator.run_evaluation(boolean_queries, regular_queries)

# Print results
def print_metrics(metrics, title):
    print(f"\n{title} Queries Evaluation Results:")
    print(f"Average Query Time: {metrics['time']:.4f}s")
    print(f"Total Relevant Documents: {metrics['total_relevant']}")
    
    print("\nPrecision@K:")
    for k in [5, 10, 20, 30]:
        print(f"@ {k}: {metrics[f'precision@{k}']:.4f}")
    
    print("\nRecall@K:")
    for k in [5, 10, 20, 30]:
        print(f"@ {k}: {metrics[f'recall@{k}']:.4f}")

print_metrics(results['boolean'], "Boolean")
print_metrics(results['regular'], "Regular")


Boolean Queries Evaluation Results:
Average Query Time: 0.0057s
Total Relevant Documents: 862

Precision@K:
@ 5: 0.0400
@ 10: 0.0400
@ 20: 0.0425
@ 30: 0.0350

Recall@K:
@ 5: 0.0070
@ 10: 0.0099
@ 20: 0.0178
@ 30: 0.0215

Regular Queries Evaluation Results:
Average Query Time: 0.0196s
Total Relevant Documents: 25570

Precision@K:
@ 5: 0.9300
@ 10: 0.9150
@ 20: 0.9050
@ 30: 0.9117

Recall@K:
@ 5: 0.0073
@ 10: 0.0138
@ 20: 0.0273
@ 30: 0.0417
